In [9]:
from __future__ import division # backward compatibility for python2
import numpy as np
import pandas as pd
import scipy.special
import operator
import random
import re


import matplotlib.pyplot as plt
%matplotlib inline

data_file = "Contact.csv"

In [10]:


#Basic Text removal code its inefficient but it works, will be improved later.
#Each Line removes the protocol settings as well as the www tag for cleaner searches which
#will be used later.
#This is part 1 of this data type where it is cleaned up for checking the emails later.

text = open("Contact.csv", "r")
text = ''.join([i for i in text]) \
    .replace("https://","")
x = open("output.csv","w")
x.writelines(text)
x.close()

text = open("output.csv", "r")
text = ''.join([i for i in text]) \
    .replace("http://","")
x = open("output.csv","w")
x.writelines(text)
x.close()

text = open("output.csv", "r")
text = ''.join([i for i in text]) \
    .replace("www.","")
x = open("output.csv","w")
x.writelines(text)
x.close()
                                           

#Loading the dataframe (df) with the cleaned up text.
df = pd.read_csv('output.csv')

#Splits the emails with split value columns
split_email = df["Email"].str.split("@",n = 1, expand = True)

#Creates new collumn names with email domain as well as email name.
df["emailname"] = split_email[0]
df["emaildomain"] = split_email[1]

#prints the dataframe to ensure that it has the correct columns
#print(df)

#Splits the website and removes the path from websites to ensure that only the hyperlink
#Remains
split_website = df["Website"].str.split("/",n= 1,expand = True)
df["Website"] = split_website[0]
#print(split_website)

#Prints to ensure the split emails are correct
#print(split_email)

df.to_csv("output.csv",index=False)
df.dropna(axis='rows')

#After this this is considered a "cleaned up" dataframe
#However common naming schemes will need to be created. 
    

,Name,Website,First,Last,Email,Phone,emailname,emaildomain
0,Kiki Hughes,kikihughes.com,Kiki,Hughes,kiki@kikihughes.com,(215)546-1534,kiki,kikihughes.com
1,DFTI Boutique,dftiboutique.com,Ashley,Taylor,ashntay@gmail.com,267-758-6790,ashntay,gmail.com
2,Boyds Philadelphia,boydsphila.com,Thomas,Carney,tcarney@boydsphila.com,(215)564-9000,tcarney,boydsphila.com
3,Smak Parlour,smakparlour.com,Abby,Kessler,smakparlour@gmail.com,(215) 625-4551,smakparlour,gmail.com
4,Knit Wit,knitwitonline.com,Anne,Gitter,info@knitwitonline.com,(215) 564-4760,info,knitwitonline.com
5,Urban Princess,urbanprincessboutique.com,Mary,Harvey,RACKANDROLL@COMCAST.NET,(267) 909-8317,RACKANDROLL,COMCAST.NET
6,Vagabond Boutique,vagabondboutique.com,Mary,Clark,vagabond.boutique@gmail.com,267-671-0737,vagabond.boutique,gmail.com
7,The Geisha House,hellogeisha.com,Cortney,Cohen Sze,thegeishahouse1@gmail.com,(267) 886-8110,thegeishahouse1,gmail.com
8,Erdon,erdon.com,Donna,Sandoz,donna@erdon.co,(215) 923-0300,donna,erdon.co
9,Addiction Designer Consigment Boutique,addictionstudios.com,Joelle,Brodhecker,jbrodhecker@yahoo.com,267.687.5446,jbrodhecker,yahoo.com


In [11]:
#Creating a new dataframe of different email naming schemes and creating
#all the potential combinations that exist

#Creating a new dataframe to be which only takes 2 columns and creates a 
#list of common email naming schemes which will then be used to check the accuracy
#of the email.
#This dataframe will be used later in the string analasys.
dfnamingscheme = df[["First","Last"]].copy()
dfnamingscheme['First'] = dfnamingscheme['First'].str.strip()
dfnamingscheme['Last'] = dfnamingscheme['Last'].str.strip()
dfnamingscheme["firstlast"] = dfnamingscheme["First"]+dfnamingscheme["Last"]
dfnamingscheme["lastfirst"] = dfnamingscheme["Last"]+dfnamingscheme["First"]
dfnamingscheme["first_last"] = dfnamingscheme["First"]+'_'+dfnamingscheme["Last"]
dfnamingscheme["first.last"] = dfnamingscheme["First"]+'.'+dfnamingscheme["Last"]
dfnamingscheme["firstletterlastname"] = dfnamingscheme["First"].str[0]+dfnamingscheme["Last"]
dfnamingscheme["firstnamelastletter"] = dfnamingscheme["First"]+dfnamingscheme["Last"].str[0]
dfnamingscheme["last.first"] = dfnamingscheme["Last"] + '.' + dfnamingscheme["First"]
dfnamingscheme["last_first"] = dfnamingscheme["Last"] + '_' + dfnamingscheme["First"]
dfnamingscheme["first-last"] = dfnamingscheme["First"] + '-' + dfnamingscheme["Last"]
dfnamingscheme["firstname-lastletter"] = dfnamingscheme["First"] + '-' + dfnamingscheme["Last"].str[0]
dfnamingscheme["firstletter-lastname"] = dfnamingscheme["First"].str[0] + '-'+ dfnamingscheme["Last"]
dfnamingscheme["firstname.lastletter"] = dfnamingscheme["First"] + '.' + dfnamingscheme["Last"].str[0]
dfnamingscheme["firstletter.lastname"] = dfnamingscheme["First"].str[0] + '.' + dfnamingscheme["Last"]
dfnamingscheme["firstletter_lastname"] = dfnamingscheme["First"].str[0] + '_' + dfnamingscheme["Last"]
dfnamingscheme["firstname_lastletter"] = dfnamingscheme["First"] + '_' + dfnamingscheme["Last"].str[0]
#For testing
#print(dfnamingscheme)
dfnamingscheme.to_csv("namingscheme.csv")

#Once all the potential email combinations are the data is cleaned up
#removing dropping
dfnamingscheme.replace(" ","")
dfnamingscheme.dropna(axis='rows')
df.dropna(axis='rows')
df.dropna(how='all', axis=1)
df.to_csv('output.csv',index=False)


        


In [12]:
#For the first check it checks the domain of the email it awards 50 accuracy if
#the website name matches the domain of the email
#Loads the previous output allows for editing and testing each code for bugs
#df = pd.read_csv('output.csv')

#Checks to see if the website is the same as the earlier email domain that was 
#Seperated from the email and creates a new column as a boolean to see if the 
#strings match.
df = pd.read_csv('output.csv')

df = df.applymap(lambda s:s.lower() if type(s) == str else s)

df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df['domainaccuracy'] = df['Website'] == df['emaildomain']
df['domainaccuracy']
#Saves the output for testing purposes
#df.to_csv('output.csv',index=False)

#Converts the booleans to a scoring system based on an int value that is set
#Currently for simplicity it is set up as 50 points.
df['domainaccuracy'] =  df['domainaccuracy'].astype(int)*50

#Printing and saving for testing
#print(df['domainaccuracy'])
df.to_csv('output.csv',index=False)



In [51]:
from __future__ import division # backward compatibility for python2
import numpy as np
import pandas as pd
import scipy.special
import operator
import random
import re


df = pd.read_csv('output.csv')
dfnamingscheme = pd.read_csv("namingscheme.csv")
#Applying a lambda map to ensure all the names are in lowercase
df = df.applymap(lambda s:s.lower() if type(s) == str else s)
dfnamingscheme = dfnamingscheme.applymap(lambda s:s.lower() if type(s) == str else s)
newtest = ""
dfnewtest = df[["emailname"]].copy()


dfnamingscheme['firstaccuracy'] = df['emailname'] == dfnamingscheme['First']
dfnamingscheme['Lastaccuracy'] = df['emailname'] == dfnamingscheme['Last']


print(dfnamingscheme['firstaccuracy'])



df.to_csv('output2.csv',index=False)

        
            


0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
17    False
18     True
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39     True
40    False
41    False
42     True
43    False
44    False
45    False
46    False
47    False
48    False
Name: firstaccuracy, dtype: bool


In [15]:
for index, row in dfnamingscheme.head().iterrows():
    if df['']
    
print(df['nameaccuracy']
      dfnamingscheme = dfnamingscheme.T
if dfnamingscheme.str.contains(df['emailname']):
      df['nameaccuracy']=True

SyntaxError: invalid syntax (<ipython-input-15-eb4d14214117>, line 2)

In [ ]:
for row1 in df['emailname']:
    found = False
    

    for row in dfnamingscheme:
        print(row)
        if row1 in row:  
            
            found = True
            newtest += "True,"
        
    if not found:
        found = False
        newtest += "False,"
        
    dfnewtest['Found'] =  found       
 